In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import regex as re
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_HGJ = pd.read_excel('/content/drive/MyDrive/Neck cancer/Raw_Data.xlsx', sheet_name='HGJ')
data_HGJ['Hospital'] ='Jewish General'
display(data_HGJ.columns)
display(data_HGJ.dtypes)
display(data_HGJ.shape)
display(data_HGJ)

In [ ]:
data_CHUS = pd.read_excel('/content/drive/MyDrive/Neck cancer/Raw_Data.xlsx', sheet_name='CHUS')
data_CHUS['Hospital'] ='Univ of Sherbrooke'
display(data_CHUS.columns)
display(data_CHUS.dtypes)
display(data_CHUS.shape)

In [ ]:
data_HMR = pd.read_excel('/content/drive/MyDrive/Neck cancer/Raw_Data.xlsx', sheet_name='HMR')
data_HMR['Hospital'] ='Maisonneuve-Rosemont'
display(data_HMR.columns)
display(data_HMR.dtypes)
display(data_HMR.shape)

In [ ]:
data_CHUM = pd.read_excel('/content/drive/MyDrive/Neck cancer/Raw_Data.xlsx', sheet_name='CHUM')
data_CHUM['Hospital'] ='Montreal Univ'
display(data_CHUM.columns)
display(data_CHUM.dtypes)
display(data_CHUM.shape)

In [ ]:
data_raw = pd.concat([data_HGJ, data_CHUS, data_HMR, data_CHUM], ignore_index=True)
display(data_raw.columns)
display(data_raw.dtypes)
display(data_raw.shape)

In [ ]:
data_raw['Time – diagnosis to diagnosis (days)'].sum()

In [ ]:
data_raw

In [ ]:
data_raw.columns

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(data_raw.isna().transpose(),
            cmap="YlGnBu",
            cbar_kws={'label': 'Missing Data'})
plt.savefig("visualizing_missing_data_with_heatmap_Seaborn_Python.png", dpi=100)

In [ ]:
col = list(data_raw.columns)
col

In [ ]:
for el in col:
    display(data_raw[el].unique())

In [ ]:
data_raw['Sex'] = data_raw['Sex'].replace({'m': 'M'})
data_raw['Age'] = data_raw['Age'].replace({77.213699: 77})
data_raw['TNM group stage'] = data_raw['TNM group stage'].replace({'stage III':'III',
                                                                   'stage IIB':'IIB',
                                                                   'stage IVA':'IVA',
                                                                   'stage IV':'IV',
                                                                   'stage IVB':'IVB',
                                                                   'stage II':'II',
                                                                   'stage I':'I',
                                                                   'Stade II':'II',
                                                                   'Stade III':'III',
                                                                   'Stade IVA':'IVA',
                                                                   'Stade I':'I',
                                                                   'Stade IVB':'IVB',
                                                                   'Stage III':'III',
                                                                   'Stage IVA':'IVA',
                                                                   'Stage IIB':'IIB',
                                                                   'Stage II':'II',
                                                                   'Stage IV':'IV',
                                                                   'StageII':'II'})

In [ ]:
data_raw.dtypes

In [ ]:
#this chart shows on the x axis, the age of the patient at entry into the study, and on the y axis the number of days
#to LR DM, or Death
df = data_raw[['Age', 'Time – diagnosis to LR (days)', 'Time – diagnosis to DM (days)', 'Time – diagnosis to Death (days)', 'Locoregional', 'Distant', 'Death', 'Therapy', 'Surgery']]
df['event'] = df[['Time – diagnosis to LR (days)','Time – diagnosis to DM (days)','Time – diagnosis to Death (days)']].min(axis=1)
df = df[df['event'].notna()]

def set_color(df):
    if df['Locoregional'] == 1:
        return 'Locoregional Metastasis'
    elif df['Distant'] == 1:
        return 'Distant Metastasis'
    else:
        return "Death"

df = df.assign(color=df.apply(set_color, axis=1))


#color="species", symbol="species"
fig = px.scatter(df, x="Age", y="event", height=1000, width=800, symbol="Death", color='color', facet_row='Therapy', labels={"Age": "Age (years)",
                                                                                                                             "event": "Time to Event (days)"},)
fig.show()

plt.savefig("scatter.png", dpi=100)

In [ ]:
df

In [ ]:
sns.set(rc = {'figure.figsize':(12,6)})
sns.set_style("white")

df_new = data_raw.groupby(['Primary Site', 'TNM group stage']).size().reset_index(name='count') #wrangle count data
df_new = df_new.pivot("Primary Site", "TNM group stage", "count")
df_new

In [ ]:
ax = sns.heatmap(df_new, annot=True, fmt='.0f', cmap="YlGnBu", linewidths=1.0, mask=(df_new==0))
plt.ylim(reversed(plt.ylim()))
ax.set_xlabel("TNM Group Stage", labelpad=15, size=13)
ax.set_ylabel("Primary Tumour Site", labelpad=15, size=13)
ax.set_title("Tumour Site vs. TNM Group Stage (n=90)", size=15, pad=15)

In [ ]:
fig = px.pie(data_raw, names='Primary Site', title='Anatomic Location of Cancerous Lesion (n=298)', hover_data=['Primary Site'], labels={'Site':'Primary Site'}, width=700, height=700, color_discrete_sequence=px.colors.sequential.RdBu_r)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
fig = px.pie(data_raw, names='T-stage', title='T-stage (n=298)', hover_data=['T-stage'], labels={'T-stage':'T-stage'},  width=700, height=700, color_discrete_sequence=px.colors.sequential.RdBu_r)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
fig = px.pie(data_raw, names='N-stage', title='N-Stage (n=298)', hover_data=['N-stage'], labels={'N-stage':'N-stage'}, width=700, height=700, color_discrete_sequence=px.colors.sequential.RdBu_r)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
fig = px.pie(data_raw, names='TNM group stage', title='TNM group stage (n=298)', hover_data=['TNM group stage'], labels={'TNM group stage':'M-stage'}, width=700, height=700, color_discrete_sequence=px.colors.sequential.RdBu_r)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
fig = px.pie(data_raw, names='HPV status', title='HPV status (n=298)', hover_data=['HPV status'], labels={'HPV status':'HPV status'}, width=700, height=700, color_discrete_sequence=px.colors.sequential.RdBu_r)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
fig = px.pie(data_raw, names='Hospital', title='Hospital (n=298)', hover_data=['Hospital'], labels={'Hospital':'Hospital'}, width=700, height=700, color_discrete_sequence=px.colors.sequential.RdBu_r)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
fig = px.histogram(data_raw, x="Age", color="Sex", title='Histogram of Age (Segregated by Sex) (n=298)', width=700, height=700, color_discrete_sequence=px.colors.sequential.RdBu_r)
fig.show()

# Data for RStudio

In [ ]:
data_raw.columns

In [ ]:
data_raw.groupby(['Primary Site']).size()

In [ ]:
#data_raw['Time – diagnosis to Death (days)'].max()

In [ ]:
SA_df = data_raw[['Patient #', 'Sex', 'Primary Site', 'T-stage', 'N-stage', 'M-stage',
                  'TNM group stage', 'HPV status', 'Time – diagnosis to last follow-up(days)',
                  'Therapy', 'Locoregional', 'Distant', 'Death', 'Time – diagnosis to LR (days)',
                  'Time – diagnosis to DM (days)', 'Time – diagnosis to Death (days)']]

#remove those with null for TNM group stage
SA_df = SA_df[SA_df['TNM group stage'].notna()]

#remove those for unknown primary site
SA_df = SA_df[(SA_df['Primary Site'] != 'unknown')]

#if event didn't occur, fill with time to last follow-up
SA_df['Time – diagnosis to LR (days)'] = SA_df['Time – diagnosis to LR (days)'].fillna(SA_df['Time – diagnosis to last follow-up(days)'])
SA_df['Time – diagnosis to DM (days)'] = SA_df['Time – diagnosis to DM (days)'].fillna(SA_df['Time – diagnosis to last follow-up(days)'])
SA_df['Time – diagnosis to Death (days)'] = SA_df['Time – diagnosis to Death (days)'].fillna(SA_df['Time – diagnosis to last follow-up(days)'])

SA_df = SA_df.rename(columns={"Time – diagnosis to LR (days)": "LR_Time",
                              "Time – diagnosis to DM (days)": "DM_Time",
                              "Time – diagnosis to Death (days)": "Death_Time"})

def set_cstage(SA_df):
    if SA_df['TNM group stage'] == 'I':
        return 'Early Stage'
    elif SA_df['TNM group stage'] == 'II':
        return 'Early Stage'
    elif SA_df['TNM group stage'] == 'IIB':
        return 'Early Stage'
    else:
        return "Late Stage"

SA_stage_df = SA_df.assign(early_late=SA_df.apply(set_cstage, axis=1))

SA_stage_df = SA_stage_df.rename(columns={"Time – diagnosis to LR (days)": "LR_Time",
                                          "Time – diagnosis to DM (days)": "DM_Time",
                                          "Time – diagnosis to Death (days)": "Death_Time"})

SA_stage_df = SA_stage_df[SA_stage_df['LR_Time'].notna()]
SA_stage_df = SA_stage_df[SA_stage_df['DM_Time'].notna()]
SA_stage_df = SA_stage_df[SA_stage_df['Death_Time'].notna()]

SA_stage_df

In [ ]:
#remove those with null for HPV status
HPV_df = SA_df[SA_df['HPV status'].notna()]

HPV_df = HPV_df[HPV_df['LR_Time'].notna()]
HPV_df = HPV_df[HPV_df['DM_Time'].notna()]
HPV_df = HPV_df[HPV_df['Death_Time'].notna()]

HPV_df

In [ ]:
def set_anat(SA_df):
    if SA_df['Primary Site'] == 'Oropharynx':
        return 'Oropharynx'
    else:
        return "Non-Oropharnyx"

SA_anat_df = SA_df.assign(anatomy=SA_df.apply(set_anat, axis=1))

SA_anat_df = SA_anat_df[SA_anat_df['LR_Time'].notna()]
SA_anat_df = SA_anat_df[SA_anat_df['DM_Time'].notna()]
SA_anat_df = SA_anat_df[SA_anat_df['Death_Time'].notna()]

SA_anat_df

In [ ]:
#SA_df.groupby(['Primary Site']).size()
#HPV_df = SA_df.groupby(['HPV status']).size()

#early vs late
#oropharynx vs anatomically lower
#HPV status + vs HPV status -

In [ ]:
#general SA stage dataframe output to csv
SA_stage_df.to_csv('SA_df.csv', encoding='utf-8', index=False)

In [ ]:
#general SA anatomy dataframe output to csv
SA_anat_df.to_csv('SA_anat_df.csv', encoding='utf-8', index=False)

In [ ]:
#filtered HPV SA dataframe output to csv
HPV_df.to_csv('HPV_SA_df.csv', encoding='utf-8', index=False)